In [13]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
from time import sleep

import http.client  # or http.client if you're on Python 3
http.client._MAXHEADERS = 10000

import warnings 
warnings.filterwarnings(action= 'ignore')

# Test

In [26]:
from selenium import webdriver

main_link = "https://www.contactcars.com/en/cars/used/engines?page=1&sortOrder=false&sortBy=CreatedAt"

chrome = webdriver.Chrome(executable_path="/Users/haitham/Downloads/chromedriver_mac64 (1)/chromedriver")



In [27]:
def get_links_list(link, driver):
    
    driver.get(link)
    scroll_down(driver)
#     z = chrome.find_elements_by_css_selector("d-block bg--white car-link")
    z = chrome.find_elements_by_class_name("car-link")

    lnk_list = []

    for i in z:
#         print(i.get_attribute("href"))
        lnk_list.append(i.get_attribute("href"))

    return lnk_list



def scroll_down(driver):
    import time
    SCROLL_PAUSE_TIME = 1.7

# Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height



def get_ad_details(l):

    car_dict = {}
# l = "https://www.contactcars.com/en/car/626997664f4dc4c0a1d6a69f-Chevrolet-Optra-1.5-A-T-H-L?status=Used"
    chrome.get(l)


    # car_details var list of WEBELEMENT include 
#     ['SUV','Body Shape','Automatic','Transmission','3,500 KM'
# ,'Mileage','Green','Color','1200 CC','Engine Capacity','Yes','Fabrica?']

    car_details = chrome.find_elements_by_class_name("detail-cards")[-1].text.split("\n")
#     print(car_details)
    # car_info var like ['Opel', 'Mokka', '2022', 'Used', 'Egypt', '9 days ago', '855,000 EGP']
    car_info = chrome.find_elements_by_class_name("used-car")[-1].text.split("\n")
#     print(car_info)

    
    print(l.split("car/")[-1].split("-")[0])
    car_dict["Ad_ID"]= l.split("car/")[-1].split("-")[0]
    
    car_dict.update(extract_details(car_details))
    
#     car_dict["Body Type"] = car_details[0]
#     car_dict["Engine Capacity (CC)"] = car_details[8]
#     car_dict["Transmission Type"] = car_details[2]
#     car_dict["Kilometers"] = car_details[4]
#     car_dict["Factory Paint"] = car_details[10]

    car_dict["Condition"] = car_info[3]
    car_dict["Year"] = car_info[2]
    car_dict["brand"] = car_info[0]
    car_dict["Model"] = car_info[1]
    car_dict["date"] =  car_info[-2]
    car_dict["price"] = car_info[-1]
    
    print(car_dict)

    return car_dict


def get_page_link(nu):
    ma ="https://www.contactcars.com/en/cars/used/engines?page="
    ink = "&sortOrder=false&sortBy=CreatedAt"
    return ma+ str(nu) + ink

def get_brand_model(string):

    import math
    s = string.split(" ")[0:-1]

    model = " ".join( s[math.floor(len(s)/2) :])
    brand = " ".join( s[: math.floor(len(s)/2)])
#     print(brand, "\n", model,"\n_______")
    return brand, model

def isFabrika(webElementList):
    fabrika = False
    for i in webElementList:
        if i.text == 'Factory Paint':
#             print("Fabrika")
            fabrika= True
        else:
            fabrika= False
    return fabrika


def extract_details(mlist):

    mdic = {}
    i = 0
    while i < len(mlist):
        if (i % 2) == 0:
            mdic[mlist[i+1]] = mlist[i]
        i+=1
        
    return mdic

In [28]:
df = pd.DataFrame()

for i in range(1,150):

    links_list =  get_links_list(get_page_link(i), chrome)

    try:
        
        for link in links_list:
            
            try:
                print("____________________")
                print(link)

                car_dict = get_ad_details(link)

                df = df.append(car_dict, ignore_index = True)
            except Exception as e2:
                print("inside exption")
                print(car_dict)
                print(e2)
                continue
    
    except Exception as e:
        print("outer excption")
        print(e)
        continue
        
    
    print(df.tail(1))
    
print("End Of Data Scrape")

____________________
https://www.contactcars.com/en/car/60076f276f6941e2a5b9f9543be5d289-Opel-Mokka-1.2-A-T-Turbo-Elegance?status=Used
60076f276f6941e2a5b9f9543be5d289
{'Ad_ID': '60076f276f6941e2a5b9f9543be5d289', 'Body Shape': 'SUV', 'Transmission': 'Automatic', 'Mileage': '3,500 KM', 'Color': 'Green', 'Engine Capacity': '1200 CC', 'Fabrica?': 'Yes', 'Condition': 'Used', 'Year': '2022', 'brand': 'Opel', 'Model': 'Mokka', 'date': '9 days ago', 'price': '855,000 EGP'}
____________________
https://www.contactcars.com/en/car/6dad670bba644b25b94a1ff53cdd8a3d-Cupra--Formentor?status=Used
6dad670bba644b25b94a1ff53cdd8a3d
{'Ad_ID': '6dad670bba644b25b94a1ff53cdd8a3d', 'Body Shape': 'Hatchback', 'Transmission': 'Automatic', 'Mileage': '100 KM', 'Engine Capacity': '1200 CC', 'Condition': 'Used', 'Year': '2023', 'brand': 'Cupra', 'Model': 'Formentor', 'date': '10 days ago', 'price': '1,400,000 EGP'}
____________________
https://www.contactcars.com/en/car/1919801aa3b34bdbb3cfd63d4fad7084-BMW-X1-1.

22df858a7da249fbad799ba297a33b3d
{'Ad_ID': '22df858a7da249fbad799ba297a33b3d', 'Body Shape': 'Sedan', 'Transmission': 'Automatic', 'Mileage': '10 KM', 'Color': 'Maroon', 'Engine Capacity': '4400 CC', 'Fabrica?': 'Yes', 'Condition': 'Used', 'Year': '2022', 'brand': 'BMW', 'Model': 'M8', 'date': '5 days ago', 'price': '3,000,000 EGP'}
____________________
https://www.contactcars.com/en/car/5db746062e2e4a049db5e1ff40e04878-BMW-430i-Coupe-2.0-A-T?status=Used


KeyboardInterrupt: 

In [ ]:
df

In [ ]:
df = df[["Ad_ID","date","Body Type","Condition", "Engine Capacity (CC)","Kilometers","Model", "Transmission Type","Year","brand","price"]]
df[100:250]

In [ ]:
df.to_csv("contact_cars2.csv")
df.shape

In [ ]:
chrome.get("https://www.contactcars.com/en/car/6283a64006683e8b604f2af1-Renault-Logan-Basic?status=Used")

In [ ]:
car_details = chrome.find_elements_by_class_name("car-details__sumary__item__details")


In [ ]:
car_details[2].text

In [ ]:
df["date"].unique()

In [ ]:
df = df.drop(df[df["date"].str.contains(pat= "Zayed", na=False, regex= True)].index)

In [12]:
chrome.quit

<bound method ChromiumDriver.quit of <selenium.webdriver.chrome.webdriver.WebDriver (session="ef8a4ce8f2f1f97b396d6dd839e80f71")>>